# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [11]:
import numpy as np

# List with all the states
states = ["V", "H", "D", "S"]
# List with all the actions
actions = ["S", "U", "D", "L", "R"]


# Computing P

h = np.array([0.6, 0.1,0.1, 0.1, 0.1]) #hare actions
p = np.zeros(shape=(5,5))


# action probabilities - ignore
p[0] = np.array([1, 0, 0, 0, 0])
p[1] = np.array([0.2, 0.8, 0, 0, 0])
p[2] = np.array([0.2, 0, 0.8, 0, 0])
p[3] = np.array([0.2, 0, 0, 0.8, 0])
p[4] = np.array([0.2, 0, 0, 0, 0.8])


P = np.zeros(shape=(5,4,4))
# s is one of (V, H, D, S)
# a is one of (0, 1, 2)
def move(s, a):
    if a == 0:
        return s
    if s == "V":
        if a == 1 or a == 2:
            return "S"
        if a == 3 or a == 4:
            return "D"
        
    if s == "H":
        if a == 1 or a == 2:
            return "D"
        if a == 3 or a == 4:
            return "S"
        
    if s == "D":
        if a == 1 or a == 2:
            return "H"
        if a == 3 or a == 4:
            return "V"
        
    if s == "S":
        if a == 1 or a == 2:
            return "V"
        if a == 3 or a == 4:
            return "H"
        
    return "ERROR"


for a in range(5):
    for s in states:
        #s = origin state
        for h_action in range(h.shape[0]):
            hare_prob = h[h_action]
            for w_action in range(p[a].shape[0]):
                wolf_prob = p[a, w_action]
                P[a, states.index(s), states.index(move(move(s, h_action), w_action))] += hare_prob*wolf_prob
    
            

# Probabilites for action stay
Pstay = P[0]
# Probabilites for action up
Pup = P[1]
# Probabilites for action down
Pdown = P[2]
# Probabilites for action left
Pleft = P[3]
# Probabilites for action right
Pright = P[4]

# Cost function
c = np.ones(shape=(len(states), len(actions)))
c[states.index('S')] = 0

print(Pup)
print(Pdown)
print(Pleft)
print(Pright)
print(Pstay)

[[ 0.28  0.16  0.04  0.52]
 [ 0.16  0.28  0.52  0.04]
 [ 0.04  0.52  0.28  0.16]
 [ 0.52  0.04  0.16  0.28]]
[[ 0.28  0.16  0.04  0.52]
 [ 0.16  0.28  0.52  0.04]
 [ 0.04  0.52  0.28  0.16]
 [ 0.52  0.04  0.16  0.28]]
[[ 0.28  0.16  0.52  0.04]
 [ 0.16  0.28  0.04  0.52]
 [ 0.52  0.04  0.28  0.16]
 [ 0.04  0.52  0.16  0.28]]
[[ 0.28  0.16  0.52  0.04]
 [ 0.16  0.28  0.04  0.52]
 [ 0.52  0.04  0.28  0.16]
 [ 0.04  0.52  0.16  0.28]]
[[ 0.6  0.   0.2  0.2]
 [ 0.   0.6  0.2  0.2]
 [ 0.2  0.2  0.6  0. ]
 [ 0.2  0.2  0.   0.6]]


### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [4]:
Policy = np.zeros(shape=(len(states), len(actions)))
# State V
Policy[states.index('V'), actions.index('U')] = 0.5
Policy[states.index('V'), actions.index('D')] = 0.5
# State H
Policy[states.index('H'), actions.index('L')] = 0.5
Policy[states.index('H'), actions.index('R')] = 0.5
# State D
Policy[states.index('D'), actions.index('U')] = 0.25
Policy[states.index('D'), actions.index('D')] = 0.25
Policy[states.index('D'), actions.index('L')] = 0.25
Policy[states.index('D'), actions.index('R')] = 0.25
#State S
Policy[states.index('S'), actions.index('S')] = 1.
print(Policy)

[[ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 1.    0.    0.    0.    0.  ]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [85]:
Ppolicy = np.zeros(shape=(4,4))
# 5x4x4 -> 4x (5x4)
# 4x5 -> 5x4
# Muk
print(np.einsum('nrm,rn->rm', P, Policy))
for s in range(len(states)):
    for a in range(len(actions)):
        Ppolicy[s, :] += Policy[s, a] * P[a, s]
Cpolicy = np.sum(c * Policy, axis=1).reshape(len(states), 1)
print(Ppolicy)

I = np.identity(4)
print( np.dot(np.dot(I, np.linalg.inv(I - 0.99*Ppolicy)), Cpolicy))

[[ 0.28  0.16  0.04  0.52]
 [ 0.16  0.28  0.04  0.52]
 [ 0.28  0.28  0.28  0.16]
 [ 0.2   0.2   0.    0.6 ]]
[[ 0.28  0.16  0.04  0.52]
 [ 0.16  0.28  0.04  0.52]
 [ 0.28  0.28  0.28  0.16]
 [ 0.2   0.2   0.    0.6 ]]
[[ 44.99671233]
 [ 44.99671233]
 [ 45.51480752]
 [ 43.88841893]]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [83]:
error = 1e-8
gama = 0.99
j = np.zeros(shape=( len(states), 1))
oldj = np.ones(shape=(len(states), 1))
def T(c, Pe, jota):
    return c + gama * np.dot(Pe, jota)
i=0
while np.sum(np.abs(oldj - j)) > error:
    oldj = j
    j = T(Cpolicy, Ppolicy, oldj)
print(j)
    

[[ 44.99671209]
 [ 44.99671209]
 [ 45.51480727]
 [ 43.88841868]]


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [5]:
# Insert your code here

### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [6]:
# Insert your code here